In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

In [2]:
import os
import re
import sklearn

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn import metrics
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

#path = './data'
path = "/Users/SarahBeadle/Documents/!WUSTL/Deep_Neural_Networks/Kaggle"

filename_test = os.path.join(path,"test.csv")
filename_train = os.path.join(path,"train.csv")
filename_sample = os.path.join(path,"sample.csv")
filename_submit = os.path.join(path,"submit.csv")
filename_checkpoint = os.path.join(path,"checkpoint.hdf5")

df_train = pd.read_csv(filename_train,na_values=['NA','?'])

np.random.seed(42) # Uncomment this line to get the same shuffle each time
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.reset_index(inplace=True, drop=True)

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
# Encode Features

# Remove all rows where the specified column is +/- sd standard deviations
def extract_and_encode_features(df):
    color_regex='(?P<color>red|blue|green|yellow|orange|pink|black|brown|white)'
    df['color'] = df.name.str.extract(color_regex, flags=re.IGNORECASE, expand=False)

    quality_regex='(?P<quality>generic|medium\shigh\squality|high\squality)'
    df['quality'] = df.name.str.extract(quality_regex, flags=re.IGNORECASE, expand=False)

    size_regex='(?P<size>tiny|small|medium|large)'
    df['size'] = df.name.str.extract(size_regex, flags=re.IGNORECASE, expand=False)

    item_regex='(?P<item>paperclips|paperweights|ink\spens|pencils|stapler|tablets|thumbtacks|post\sit\snotes)'
    df['item'] = df.name.str.extract(item_regex, flags=re.IGNORECASE, expand=False)
    
    for column in ['pack','weight','height','width','length']:
        missing_median(df,column)
    

extract_and_encode_features(df_train)
ids_train = df_train['id']
df_train.drop('id',1,inplace=True)

names_train = df_train['name']
df_train.drop('name',1,inplace=True)

In [4]:
missing_default(df_train, "quality", "Unknown")
missing_default(df_train, "size", "Unknown")
missing_default(df_train, "color", "Unknown")

In [5]:
# encode text/categorical features
for column in ['manufacturer','color','quality','size','item']:
    encode_text_dummy(df_train,column)

In [27]:
x,y = to_xy(df_train,'cost')
# Cross-Validate
#kf = KFold(5)

# Used before KFold
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=45)

#oos_y = []
#oos_pred = []
#fold = 0
checkpoint = ModelCheckpoint(filepath=filename_checkpoint, verbose=0, save_best_only=True)
#for train, test in kf.split(x):
#    fold+=1
#    print("Fold #{}".format(fold))
        
#x_train = x[train]
#y_train = y[train]
#x_test = x[test]
#y_test = y[test]

#changed dense values from (50, 25, 10) to (45, 30, 15), patience = 100
#original RMSE was 13.15, new value is 11.87, 2nd RMSE is 11.057, results kagglecode.csv
#2nd RMSE changed 3rd layer to 10 (45, 30, 10)
#1st RMSE is 11.75, 2nd RMSE is 11.10, results kagglecode2.csv
#3rd try, (45,30,10), changed patience from 100 to 50
#1st RMSE is 13.65, 11.49, did not save results
#4th try, (45,30,10), changed patience to 150
#1st RMSE is 11.18, 2nd RMSE is 10.94, results kagglesubmission.csv


model = Sequential()
model.add(Dense(45, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dropout(0.01)) # Dropout Layer
model.add(Dense(30, activation='relu')) # Hidden 2
model.add(Dense(10, 
            kernel_regularizer=regularizers.l2(0.01),
            activity_regularizer=regularizers.l1(0.01),activation='relu')) # Hidden 3 w/regularization
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=150, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpoint],verbose=1,epochs=1000)
    
pred = model.predict(x_test)
    
    #oos_y.append(y_test)
    #oos_pred.append(pred)        

    # Measure this fold's RMSE
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Fold score (RMSE): {}".format(score))

Train on 7500 samples, validate on 2500 samples
Epoch 1/1000
7500/7500 [==============================] - 1s 166us/step - loss: 5087.5010 - val_loss: 4384.2793
Epoch 2/1000
7500/7500 [==============================] - 1s 72us/step - loss: 4682.2438 - val_loss: 4327.9726
Epoch 3/1000
7500/7500 [==============================] - 1s 72us/step - loss: 4530.1508 - val_loss: 4146.3087
Epoch 4/1000
7500/7500 [==============================] - 1s 77us/step - loss: 4290.0310 - val_loss: 4061.9453
Epoch 5/1000
7500/7500 [==============================] - 1s 67us/step - loss: 4211.9335 - val_loss: 3810.2652
Epoch 6/1000
7500/7500 [==============================] - 0s 60us/step - loss: 4263.3557 - val_loss: 4100.0885
Epoch 7/1000
7500/7500 [==============================] - 0s 66us/step - loss: 4077.9316 - val_loss: 4223.8632
Epoch 8/1000
7500/7500 [==============================] - 1s 76us/step - loss: 4059.6892 - val_loss: 3750.6401
Epoch 9/1000
7500/7500 [==============================] - 0s 65

7500/7500 [==============================] - 0s 63us/step - loss: 340.9654 - val_loss: 346.6682
Epoch 75/1000
7500/7500 [==============================] - 1s 69us/step - loss: 417.5067 - val_loss: 597.4543
Epoch 76/1000
7500/7500 [==============================] - 1s 71us/step - loss: 407.4519 - val_loss: 332.7393
Epoch 77/1000
7500/7500 [==============================] - 1s 77us/step - loss: 375.7521 - val_loss: 241.0559
Epoch 78/1000
7500/7500 [==============================] - 0s 66us/step - loss: 367.8156 - val_loss: 270.5881
Epoch 79/1000
7500/7500 [==============================] - 0s 64us/step - loss: 320.8260 - val_loss: 522.5417
Epoch 80/1000
7500/7500 [==============================] - 1s 72us/step - loss: 365.6131 - val_loss: 263.9486
Epoch 81/1000
7500/7500 [==============================] - 0s 62us/step - loss: 371.5362 - val_loss: 244.1264
Epoch 82/1000
7500/7500 [==============================] - 0s 64us/step - loss: 405.8181 - val_loss: 234.4090
Epoch 83/1000
7500/7500 

7500/7500 [==============================] - 0s 59us/step - loss: 262.3094 - val_loss: 296.0117
Epoch 149/1000
7500/7500 [==============================] - 0s 58us/step - loss: 263.8046 - val_loss: 238.1942
Epoch 150/1000
7500/7500 [==============================] - 0s 59us/step - loss: 279.1620 - val_loss: 395.5625
Epoch 151/1000
7500/7500 [==============================] - 0s 59us/step - loss: 270.0057 - val_loss: 221.4589
Epoch 152/1000
7500/7500 [==============================] - 0s 60us/step - loss: 292.9482 - val_loss: 247.5808
Epoch 153/1000
7500/7500 [==============================] - 0s 57us/step - loss: 300.0999 - val_loss: 225.6520
Epoch 154/1000
7500/7500 [==============================] - 0s 62us/step - loss: 248.6700 - val_loss: 197.1946
Epoch 155/1000
7500/7500 [==============================] - 0s 58us/step - loss: 254.9199 - val_loss: 451.0877
Epoch 156/1000
7500/7500 [==============================] - 0s 58us/step - loss: 397.9315 - val_loss: 389.3257
Epoch 157/1000
7

7500/7500 [==============================] - 0s 58us/step - loss: 225.4027 - val_loss: 208.5243
Epoch 222/1000
7500/7500 [==============================] - 0s 63us/step - loss: 231.6611 - val_loss: 168.0730
Epoch 223/1000
7500/7500 [==============================] - 0s 59us/step - loss: 247.3820 - val_loss: 219.0396
Epoch 224/1000
7500/7500 [==============================] - 0s 58us/step - loss: 258.8814 - val_loss: 211.5883
Epoch 225/1000
7500/7500 [==============================] - 0s 60us/step - loss: 295.1194 - val_loss: 300.4324
Epoch 226/1000
7500/7500 [==============================] - 0s 59us/step - loss: 297.4882 - val_loss: 173.0760
Epoch 227/1000
7500/7500 [==============================] - 0s 63us/step - loss: 210.4346 - val_loss: 164.7683
Epoch 228/1000
7500/7500 [==============================] - 0s 60us/step - loss: 298.0920 - val_loss: 169.7123
Epoch 229/1000
7500/7500 [==============================] - 0s 60us/step - loss: 220.6544 - val_loss: 172.3742
Epoch 230/1000
7

Epoch 295/1000
7500/7500 [==============================] - 0s 58us/step - loss: 213.4310 - val_loss: 176.1030
Epoch 296/1000
7500/7500 [==============================] - 0s 58us/step - loss: 353.7564 - val_loss: 212.2841
Epoch 297/1000
7500/7500 [==============================] - 0s 58us/step - loss: 239.7922 - val_loss: 346.4353
Epoch 298/1000
7500/7500 [==============================] - 0s 60us/step - loss: 240.3225 - val_loss: 334.9918
Epoch 299/1000
7500/7500 [==============================] - 0s 57us/step - loss: 250.9488 - val_loss: 1190.7646
Epoch 300/1000
7500/7500 [==============================] - 0s 59us/step - loss: 386.5031 - val_loss: 460.4032
Epoch 301/1000
7500/7500 [==============================] - 0s 62us/step - loss: 263.7918 - val_loss: 159.4532
Epoch 302/1000
7500/7500 [==============================] - 0s 60us/step - loss: 214.4598 - val_loss: 178.6758
Epoch 303/1000
7500/7500 [==============================] - 0s 63us/step - loss: 220.3004 - val_loss: 158.9602


...

Epoch 516/1000
7500/7500 [==============================] - 0s 53us/step - loss: 178.1266 - val_loss: 163.1918
Epoch 517/1000
7500/7500 [==============================] - 0s 53us/step - loss: 196.7915 - val_loss: 164.6589
Epoch 518/1000
7500/7500 [==============================] - 0s 53us/step - loss: 229.5012 - val_loss: 156.7872
Epoch 519/1000
7500/7500 [==============================] - 0s 52us/step - loss: 194.2822 - val_loss: 154.4416
Epoch 520/1000
7500/7500 [==============================] - 0s 53us/step - loss: 195.5503 - val_loss: 166.1364
Epoch 521/1000
7500/7500 [==============================] - 0s 52us/step - loss: 193.8960 - val_loss: 177.8432
Epoch 522/1000
7500/7500 [==============================] - 0s 54us/step - loss: 176.7614 - val_loss: 251.9194
Epoch 523/1000
7500/7500 [==============================] - 0s 53us/step - loss: 229.7675 - val_loss: 172.4320
Epoch 524/1000
7500/7500 [==============================] - 0s 54us/step - loss: 166.5684 - val_loss: 172.0457
E

Epoch 590/1000
7500/7500 [==============================] - 0s 53us/step - loss: 176.2216 - val_loss: 150.6400
Epoch 591/1000
7500/7500 [==============================] - 0s 53us/step - loss: 186.0364 - val_loss: 171.4729
Epoch 592/1000
7500/7500 [==============================] - 0s 53us/step - loss: 200.8888 - val_loss: 218.7462
Epoch 593/1000
7500/7500 [==============================] - 0s 53us/step - loss: 187.1539 - val_loss: 153.1138
Epoch 594/1000
7500/7500 [==============================] - 0s 53us/step - loss: 187.3103 - val_loss: 143.4687
Epoch 595/1000
7500/7500 [==============================] - 0s 53us/step - loss: 205.3646 - val_loss: 216.9109
Epoch 596/1000
7500/7500 [==============================] - 0s 53us/step - loss: 177.2028 - val_loss: 164.9105
Epoch 597/1000
7500/7500 [==============================] - 0s 53us/step - loss: 165.3974 - val_loss: 151.4011
Epoch 598/1000
7500/7500 [==============================] - 0s 53us/step - loss: 168.5085 - val_loss: 169.8282
E

Epoch 664/1000
7500/7500 [==============================] - 0s 53us/step - loss: 188.5072 - val_loss: 145.6126
Epoch 665/1000
7500/7500 [==============================] - 0s 53us/step - loss: 176.3982 - val_loss: 153.3231
Epoch 666/1000
7500/7500 [==============================] - 0s 53us/step - loss: 163.3039 - val_loss: 153.7898
Epoch 667/1000
7500/7500 [==============================] - 0s 53us/step - loss: 177.3621 - val_loss: 143.0542
Epoch 668/1000
7500/7500 [==============================] - 0s 53us/step - loss: 235.1766 - val_loss: 145.5065
Epoch 669/1000
7500/7500 [==============================] - 0s 54us/step - loss: 169.1045 - val_loss: 146.6605
Epoch 670/1000
7500/7500 [==============================] - 0s 53us/step - loss: 170.0993 - val_loss: 220.3758
Epoch 671/1000
7500/7500 [==============================] - 0s 52us/step - loss: 173.9937 - val_loss: 148.0949
Epoch 672/1000
7500/7500 [==============================] - 0s 57us/step - loss: 181.7351 - val_loss: 141.6720
E

Epoch 738/1000
7500/7500 [==============================] - 0s 52us/step - loss: 158.1197 - val_loss: 156.4657
Epoch 739/1000
7500/7500 [==============================] - 0s 53us/step - loss: 157.2572 - val_loss: 187.3909
Epoch 740/1000
7500/7500 [==============================] - 0s 52us/step - loss: 162.7132 - val_loss: 186.0432
Epoch 741/1000
7500/7500 [==============================] - 0s 52us/step - loss: 176.8395 - val_loss: 140.0698
Epoch 742/1000
7500/7500 [==============================] - 0s 53us/step - loss: 198.8727 - val_loss: 429.7610
Epoch 743/1000
7500/7500 [==============================] - 0s 53us/step - loss: 211.9016 - val_loss: 154.2210
Epoch 744/1000
7500/7500 [==============================] - 0s 53us/step - loss: 160.5902 - val_loss: 146.1758
Epoch 745/1000
7500/7500 [==============================] - 0s 53us/step - loss: 167.7024 - val_loss: 510.2064
Epoch 746/1000
7500/7500 [==============================] - 0s 53us/step - loss: 178.5053 - val_loss: 152.6840
E

Epoch 812/1000
7500/7500 [==============================] - 0s 64us/step - loss: 164.3240 - val_loss: 159.9621
Epoch 813/1000
7500/7500 [==============================] - 0s 64us/step - loss: 235.5987 - val_loss: 162.0194
Epoch 814/1000
7500/7500 [==============================] - 0s 65us/step - loss: 163.1835 - val_loss: 184.8222
Epoch 815/1000
7500/7500 [==============================] - 0s 63us/step - loss: 186.5259 - val_loss: 299.2153
Epoch 816/1000
7500/7500 [==============================] - 0s 66us/step - loss: 175.7810 - val_loss: 143.2499
Epoch 817/1000
7500/7500 [==============================] - 1s 72us/step - loss: 165.1533 - val_loss: 139.3245
Epoch 818/1000
7500/7500 [==============================] - 1s 71us/step - loss: 170.5460 - val_loss: 192.0367
Epoch 819/1000
7500/7500 [==============================] - 1s 70us/step - loss: 187.6504 - val_loss: 266.3229
Epoch 820/1000
7500/7500 [==============================] - 1s 73us/step - loss: 162.8807 - val_loss: 140.7993
E

In [29]:
model.load_weights(filename_checkpoint)
pred = model.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

#chart_regression(pred.flatten(),y_test)
#chart_regression(pred.flatten(),y_test,sort=False)

Final score (RMSE): 10.943763732910156


In [30]:
df_test = pd.read_csv(filename_test,na_values=['NA','?'])

extract_and_encode_features(df_test)
missing_default(df_test, "quality", "Unknown")
missing_default(df_test, "size", "Unknown")
missing_default(df_test, "color", "Unknown")

for column in ['manufacturer','color','quality','size','item']:
    encode_text_dummy(df_test,column)

ids_test = df_test['id']
df_test.drop('id',1,inplace=True)

names_test = df_test['name']
df_test.drop('name',1,inplace=True)

x_submit = df_test.as_matrix().astype(np.float32)
pred_submit = model.predict(x_submit)

In [31]:
df_submit = pd.DataFrame(pred_submit)
df_submit.insert(0,'id',ids_test)
df_submit.columns = ['id','cost']

In [33]:
df_submit.to_csv('kagglesubmission.csv', index=False)